In [ ]:
pip install torch torchvision matplotlib

In [ ]:
from torchvision import datasets
from torchvision.transforms import ToTensor
from sklearn.metrics import precision_score, recall_score


In [ ]:
train_data = datasets.MNIST(
    root = 'data',
    train = True ,
    transform = ToTensor(),
    download = True
    )


test_data = datasets.MNIST(
    root = 'data',
    train = False ,
    transform = ToTensor(),
    download = True
    )

In [ ]:
from torch.utils.data import DataLoader

loaders = {
    'train': DataLoader(train_data, batch_size=100, shuffle=True, num_workers=1),
    'test': DataLoader(test_data, batch_size=100, shuffle=True, num_workers=1),
}

In [ ]:
loaders


In [ ]:
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

class CNN(nn.Module):

  def __init__(self):
    super(CNN, self).__init__()

    self.conv1 = nn.Conv2d(1, 10, kernel_size=5)
    self.conv2 = nn.Conv2d(10, 20, kernel_size=5)
    self.conv2_drop = nn.Dropout2d() #normalization, deactivates layer
    self.fc1 = nn.Linear(320, 50)
    self.fc2 = nn.Linear(50, 10) # 10 because 10 digits and softmax function will be used - prob for every digit

  def forward(self, x):
    x = F.relu(F.max_pool2d(self.conv1(x), 2))
    x = F.relu(F.max_pool2d(self.conv2_drop(self.conv2(x)), 2))
    x = x.view(-1, 320)
    x = F.relu(self.fc1(x))
    x = F.dropout(x, training=self.training)
    x = self.fc2(x)

    return F.softmax(x, dim=1)


In [ ]:
import torch

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

model = CNN().to(device)

optimizer = optim.Adam(model.parameters(), lr=0.001)

loss_fn = nn.CrossEntropyLoss()

def train(epoch):
  model.train()
  for batch_idx, (data, target) in enumerate(loaders['train']):
    data, target = data.to(device), target.to(device)
    optimizer.zero_grad()
    output = model(data)
    loss = loss_fn(output, target)
    loss.backward()
    optimizer.step()

    if batch_idx % 50 == 0 :
      print(f'Train Epoch: {epoch} [{batch_idx * len(data)}/{len(loaders["train"].dataset)} ({100. * batch_idx / len(loaders["train"]):.0f}%)]\t{loss.item():.6f}')


def test():
  model.eval()

  test_loss = 0
  correct = 0

  all_preds = []
  all_targets = []

  with torch.no_grad():
    for data, target in loaders['test']:
      data, target = data.to(device), target.to(device)
      output = model(data)
      test_loss += loss_fn(output, target).item()
      pred = output.argmax(dim=1, keepdim = True)
      correct += pred.eq(target.view_as(pred)).sum().item()

      all_preds.extend(pred.cpu().numpy())
      all_targets.extend(target.cpu().numpy())

  precision = precision_score(all_targets, all_preds, average='macro')
  recall = recall_score(all_targets, all_preds, average='macro')

  print(f'Precision: {precision:.4f}, Recall: {recall:.4f}')

  test_loss /= len(loaders['test'].dataset)
  print(f'\nTest set: Average loss: {test_loss: .4f}, Accuracy {correct}/{len(loaders["test"].dataset)} ({100. * correct/len(loaders["test"].dataset):.0f}%\n)')


In [ ]:
for epoch in range(1, 11):
  train(epoch)
  test()

In [ ]:
import os
import torch
import torchvision.transforms as transforms
from PIL import Image

# Path to the folder containing custom handwritten digit images
custom_data_path = "my_digits/"

# Image transformation to match MNIST format (28x28, grayscale, tensor)
transform = transforms.Compose([
    transforms.Grayscale(num_output_channels=1),  # Ensure image is in grayscale
    transforms.Resize((28, 28)),                  # Resize to 28x28 pixels
    transforms.ToTensor(),                        # Convert image to tensor
    transforms.Normalize((0.1307,), (0.3081,))     # Normalize as in MNIST
])

# Lists to store images and labels
custom_images = []
custom_labels = []

# Debug: list files in the folder
files_in_dir = os.listdir(custom_data_path)
print("Files found:", files_in_dir)

# Iterate through all PNG files in the folder
for file in files_in_dir:
    if file.lower().endswith(".png"):
        img_path = os.path.join(custom_data_path, file)
        try:
            # Open and transform the image
            img = Image.open(img_path)
            img = transform(img)
            custom_images.append(img)

            # Extract label from the filename (e.g., "0.1.png" -> label 0)
            label = int(file.split('.')[0])
            custom_labels.append(label)
        except Exception as e:
            print(f"Error loading {img_path}: {e}")

# Ensure that images were successfully loaded
if not custom_images:
    raise RuntimeError("No images were loaded. Please verify the folder path and file formats.")

# Convert lists to PyTorch tensors
custom_images = torch.stack(custom_images)
custom_labels = torch.tensor(custom_labels)

print(f"Loaded {len(custom_images)} custom handwritten digit images.")

# Set the model to evaluation mode (assuming `model` and `device` are already defined)
model.eval()

# Move custom images and labels to the same device as the model
custom_images = custom_images.to(device)
custom_labels = custom_labels.to(device)

# Obtain predictions from the model
with torch.no_grad():
    outputs = model(custom_images)
    predictions = outputs.argmax(dim=1)

# Calculate accuracy
correct = (predictions == custom_labels).sum().item()
accuracy = 100. * correct / len(custom_labels)
print(f"Recognition accuracy on custom handwritten digits: {accuracy:.2f}%")

# Analyze and print misclassifications
for i in range(len(custom_labels)):
    true_label = custom_labels[i].item()
    predicted_label = predictions[i].item()
    if true_label != predicted_label:
        print(f"🔴 ERROR: True digit: {true_label}, Model predicted: {predicted_label}")


In [ ]:
import numpy as np
from torchvision import datasets, transforms
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score

# Load MNIST dataset using torchvision
train_dataset = datasets.MNIST(
    root='data',
    train=True,
    transform=transforms.ToTensor(),
    download=True
)
test_dataset = datasets.MNIST(
    root='data',
    train=False,
    transform=transforms.ToTensor(),
    download=True
)

# Convert dataset images and labels to numpy arrays.
# Flatten each 28x28 image to a vector of size 784.
X_train = train_dataset.data.numpy().reshape(-1, 28*28)
y_train = train_dataset.targets.numpy()
X_test = test_dataset.data.numpy().reshape(-1, 28*28)
y_test = test_dataset.targets.numpy()

# Create and train the Random Forest classifier
rf = RandomForestClassifier(n_estimators=100, random_state=42, n_jobs=-1)
rf.fit(X_train, y_train)

# Predict on the test set
y_pred = rf.predict(X_test)

# Compute performance metrics
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred, average='macro')
recall = recall_score(y_test, y_pred, average='macro')

print(f"Recognition Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Sensitivity (Recall): {recall:.4f}")
